In [8]:
### 기본 라이브러리 불러오기
import pandas as pd
import seaborn as sns

'''
[Step 1] 데이터 준비 - Seaborn에서 제공하는 titanic 데이터셋 가져오기
'''

# load_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')

# 데이터 살펴보기


#  IPython 디스플레이 설정 - 출력할 열의 개수 한도 늘리기
pd.set_option('display.max_columns', 15)



'''
[Step 2] 데이터 탐색
'''

# 데이터 자료형 확인


# NaN값이 많은 deck 열을 삭제, embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)  


# # age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
# rdf = rdf.dropna(subset=['age'], how='any', axis=0)  
# print(len(rdf))
# print('\n')

# age 결측치 채워주기- 평균값 활용
rdf['age'].fillna(value=rdf['age'].mean(), inplace=True)



# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()   


rdf['embarked'].fillna(most_freq, inplace=True)



'''
[Step 3] 분석에 사용할 속성을 선택
'''

# 분석에 활용할 열(속성)을 선택 
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]
print(ndf.head())   
print('\n')

# 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex', 'embarked','female','town_S'], axis=1, inplace=True)



'''
[Step 4] 데이터셋 구분 - 훈련용(train data)/ 검증용(test data)
'''

# 속성(변수) 선택
X=ndf[['pclass', 'age', 'sibsp', 'parch', 'male', 
       'town_C', 'town_Q']]  #독립 변수 X
y=ndf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 




   survived  pclass     sex   age  sibsp  parch embarked
0         0       3    male  22.0      1      0        S
1         1       1  female  38.0      1      0        C
2         1       3  female  26.0      0      0        S
3         1       1  female  35.0      1      0        S
4         0       3    male  35.0      0      0        S




In [9]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(criterion='entropy', 
                             n_estimators=2000,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
model_rf.fit(X_train, y_train)
y_hat = model_rf.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

print("%.4f" % model_rf.oob_score_)
print("%.5f" %model_rf.oob_score_)

[0 0 0 1 1 0 0 0 0 0]
[0 0 0 1 1 0 0 0 1 0]
0.8058
0.80578


In [10]:
# model 성능확인
from sklearn import metrics as mt

y_hat_t=model_rf.predict(X_train)

t_acc = mt.accuracy_score(y_train, y_hat_t)
acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)

print("train accuracy", t_acc)
print("accuracy", acc )
print("confusion matrix\n",conf)

# 모형 성능 평가 - 평가지표 계산
report_rf = mt.classification_report(y_test, y_hat)            
print(report_rf)

train accuracy 0.8780096308186196
accuracy 0.835820895522388
confusion matrix
 [[156  18]
 [ 26  68]]
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       174
           1       0.79      0.72      0.76        94

    accuracy                           0.84       268
   macro avg       0.82      0.81      0.82       268
weighted avg       0.83      0.84      0.83       268



In [11]:
# variable importance
ndf_ = ndf[['pclass', 'age', 'sibsp', 'parch', 'male', 
       'town_C', 'town_Q']]

pd.concat((pd.DataFrame(ndf_.columns, columns = ['variable']), 
           pd.DataFrame(model_rf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

# model_rf.feature_importances_

,variable,importance
4,male,0.339397
1,age,0.321635
0,pclass,0.137181
2,sibsp,0.094479
3,parch,0.057073
5,town_C,0.031994
6,town_Q,0.018241
